In [9]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from DrawtexModel import DrawTexModel
from DrawtexDataset import DrawtexDataset

#### Hyperparameters

In [10]:
TRAIN_BATCH_SZ = 128
TEST_BATCH_SZ = 1000
EPOCHS = 10
LEARN_RATE = 0.01
CLASS_CNT = 82

device: str = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


#### Dataloader setup

In [11]:
data_set = DrawtexDataset(transforms.ToTensor())
TRAIN_SIZE = int(0.8 * len(data_set))
TEST_SIZE = len(data_set) - TRAIN_SIZE
train_set, test_set = torch.utils.data.random_split(data_set, [TRAIN_SIZE, TEST_SIZE])

train_loader = DataLoader(
    dataset=train_set,
    batch_size=TRAIN_BATCH_SZ,
    shuffle=True,
    num_workers=4
)

test_loader = DataLoader(
    dataset=test_set,
    batch_size=TEST_BATCH_SZ,
    shuffle=False,
    num_workers=4
)

model = DrawTexModel().to(device)
print(model)

DrawTexModel(
  (relu): ReLU()
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(3, 3), stride=(1, 1))
  (lin1): Linear(in_features=30420, out_features=500, bias=True)
  (lin2): Linear(in_features=500, out_features=82, bias=True)
)


In [12]:
def train():
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=LEARN_RATE)
    steps = len(train_loader)
    for epoch in range(EPOCHS):
        for i, (img, label) in enumerate(train_loader):
            img: torch.Tensor = img.to(device, non_blocking=True)
            label: torch.Tensor = label.to(device, non_blocking=True)
            output = model(img)
            loss: torch.Tensor = criterion(output, label).to(device)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (i + 1) % TRAIN_BATCH_SZ == 0:
                print(f"Epoch {epoch}/{EPOCHS}, Step {i + 1}/{steps}, Loss {loss.item():.4f}")

train()
torch.save(model.state_dict(), "./DrawTexModel.pth")


Epoch 0/10, Step 128/2350, Loss 3.7226
Epoch 0/10, Step 256/2350, Loss 3.6455
Epoch 0/10, Step 384/2350, Loss 3.8262
Epoch 0/10, Step 512/2350, Loss 3.7792
Epoch 0/10, Step 640/2350, Loss 3.5864
Epoch 0/10, Step 768/2350, Loss 3.8210
Epoch 0/10, Step 896/2350, Loss 3.7410
Epoch 0/10, Step 1024/2350, Loss 3.6869
Epoch 0/10, Step 1152/2350, Loss 3.6522
Epoch 0/10, Step 1280/2350, Loss 3.7225
Epoch 0/10, Step 1408/2350, Loss 3.6468
Epoch 0/10, Step 1536/2350, Loss 3.7665
Epoch 0/10, Step 1664/2350, Loss 3.6963
Epoch 0/10, Step 1792/2350, Loss 3.6438
Epoch 0/10, Step 1920/2350, Loss 3.7650
Epoch 0/10, Step 2048/2350, Loss 3.6921
Epoch 0/10, Step 2176/2350, Loss 3.6139
Epoch 0/10, Step 2304/2350, Loss 3.7207
Epoch 1/10, Step 128/2350, Loss 3.7727
Epoch 1/10, Step 256/2350, Loss 3.5014
Epoch 1/10, Step 384/2350, Loss 3.6687
Epoch 1/10, Step 512/2350, Loss 3.7680
Epoch 1/10, Step 640/2350, Loss 3.5057
Epoch 1/10, Step 768/2350, Loss 3.5137
Epoch 1/10, Step 896/2350, Loss 3.6130
Epoch 1/10, St

In [13]:
with torch.no_grad():
    correct = 0
    total = 0
    for img, labels in test_loader:
        img = img.to(device, non_blocking= True)
        labels = labels.to(device, non_blocking= True)

        output = model(img)
        _, prediction = torch.max(output, 1)
        total += labels.size(0)
        correct += (prediction == labels).sum().item()

    acc = 100.0 * correct / total
    print(f"Accuracy: {acc}%")

Accuracy: 7.820998736618126%
